# [Module 2.4] DeepAR+ 및 Prophet Predictor 생성 
- 여기서는 두개의 Predictos 인 Prophet 과 DeepAR+를 생성 (포케스팅 모델 학습) 을 하고, 실제 Predictor의 성능 지표를 통해서 얼마나 성능이 나왔는지를 학인 할 수 있습니다.
* 이 과정은 약 40분 정도 걸립니다. **About 40 mins may be elapsed**



In [1]:
import boto3
from time import sleep
import pandas as pd
import json
import time
import pprint
import numpy as np


In [2]:
%store -r

In [3]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')

## Parameters

- forecastHorizon 는 13 개의 미래 데이타 포인트를 예측 합니다. ForecastFrequency 가 Week 기준이므로 미래의 13주를 예측 합니다.
- NumberOfBacktestWindows 는 백테스트를 의미합니다. 학습 데이타의 맨 마지막에서 2번 백테스트 합니다.

In [4]:
forecastHorizon = 13 # the number of forecasting
NumberOfBacktestWindows = 2 # the number of Backtests
BackTestWindowOffset = 13 # the point from the end of the dataset where the data is split for model training and testing 
ForecastFrequency = "W"

A detailed parameter info is [here](https://docs.aws.amazon.com/forecast/latest/dg/metrics.html)

#### Predictor algorithms

In [5]:
prophet_algorithmArn = 'arn:aws:forecast:::algorithm/Prophet'
deepAR_Plus_algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

### Prophet

In [6]:
# Prophet Specifics
prophet_predictorName= project+'_prophet_algo_1' + item_meta_suffix + suffix

In [7]:
# Build Prophet:
prophet_create_predictor_response=forecast.create_predictor(PredictorName=prophet_predictorName, 
                                                  AlgorithmArn=prophet_algorithmArn,
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML= False,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                                                                         "BackTestWindowOffset": BackTestWindowOffset}, 
                                                  InputDataConfig= {"DatasetGroupArn": item_meta_datasetGroupArn, "SupplementaryFeatures": [ 
                                                                     { 
                                                                        "Name": "holiday",
                                                                        "Value": "US"
                                                                     }
                                                                  ]},
                                                  FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "target_value", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )




### DeepAR+

In [8]:
# DeepAR+ Specifics
deeparp_predictorName= project+'_deeparp_algo_1' + item_meta_suffix + suffix

In [9]:
# Build DeepAR+:
deeparp_create_predictor_response=forecast.create_predictor(PredictorName=deeparp_predictorName, 
                                                  AlgorithmArn=deepAR_Plus_algorithmArn,
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML= False,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                                                                         "BackTestWindowOffset": BackTestWindowOffset}, 
                                                  InputDataConfig= {"DatasetGroupArn": item_meta_datasetGroupArn, "SupplementaryFeatures": [ 
                                                                     { 
                                                                        "Name": "holiday",
                                                                        "Value": "US"
                                                                     }
                                                                  ]},
                                                  FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "target_value", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )





## Check if the predictors are active


In [10]:
# forecast.list_predictors(Filters=[
#     {"Condition": "IS",
#      "Key": "DatasetGroupArn", 
#      "Value": datasetGroupArn}
# ])

In [11]:

item_meta_prophet_predictor_arn = prophet_create_predictor_response["PredictorArn"]
item_meta_deepar_predictor_arn = deeparp_create_predictor_response["PredictorArn"]

In [12]:
%%time
# Check the Prophet status

while True:
    createProphetPredictorStatus = forecast.describe_predictor(PredictorArn= item_meta_prophet_predictor_arn)['Status']
    createDeeparpPredictorStatus = forecast.describe_predictor(PredictorArn= item_meta_deepar_predictor_arn)['Status']    
    print("Prophet: ", createProphetPredictorStatus)
    print("DeepARP: ", createDeeparpPredictorStatus)          
    if createProphetPredictorStatus != 'ACTIVE' and createProphetPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpPredictorStatus != 'ACTIVE' and createDeeparpPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

Prophet:  CREATE_PENDING
DeepARP:  CREATE_PENDING
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PR

## Predictor evaluation

#### Prophet

In [13]:
# Prophet Metrics
prophet_metrics = forecast.get_accuracy_metrics(PredictorArn=item_meta_prophet_predictor_arn)
pp = pprint.PrettyPrinter(indent=1)
pp.pprint(prophet_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Prophet',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 102825.48601500283,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.03622294053008564,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.06994082553756105,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.03260350019995557,
                                                                                          'Quantile': 0.1}]}},
                                           

#### DeepAR+

In [14]:
# DeepAR+ Metrics
deeparp_metrics = forecast.get_accuracy_metrics(PredictorArn=item_meta_deepar_predictor_arn)
pp = pprint.PrettyPrinter()
pp.pprint(deeparp_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 97744.1946411623,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.032038099838833545,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.06333066063737518,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.02941662916391388,
                                                                                          'Quantile': 0.1}]}},
                                       

In [15]:
%store item_meta_prophet_predictor_arn
%store item_meta_deepar_predictor_arn

Stored 'item_meta_prophet_predictor_arn' (str)
Stored 'item_meta_deepar_predictor_arn' (str)
